# CXRlabeler Model

# PadChest Dataset 

In [26]:
from fastai.basics import *
from fastai.text.all import *
import warnings
warnings.filterwarnings('ignore')
# Read in the train and test sets.
path = Path('/home/jupyter/data/padchest')
df_lm = pd.read_csv(path/"lm.csv")
df_cl = pd.read_csv(path/"labels.csv")
df_train = pd.read_csv(path/"train.csv")
df_test = pd.read_csv(path/"test.csv")

In [27]:
df_lm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29365 entries, 0 to 29364
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ImageID  29365 non-null  object
 1   Report   29365 non-null  object
dtypes: object(2)
memory usage: 459.0+ KB


In [28]:
df_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29365 entries, 0 to 29364
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ImageID           29365 non-null  object
 1   Report            29365 non-null  object
 2   Atelectasis       29365 non-null  int64 
 3   Cardiomegaly      29365 non-null  int64 
 4   Consolidation     29365 non-null  int64 
 5   Edema             29365 non-null  int64 
 6   Fracture          29365 non-null  int64 
 7   No Finding        29365 non-null  int64 
 8   Pleural Effusion  29365 non-null  int64 
 9   Pneumonia         29365 non-null  int64 
 10  Pneumothorax      29365 non-null  int64 
 11  is_valid          29365 non-null  bool  
dtypes: bool(1), int64(9), object(2)
memory usage: 2.5+ MB


# 1. Language Model

In [29]:
df_text = pd.Series.append(df_train['Report'], df_test['Report'])

In [30]:
df_text = pd.DataFrame(df_text)

In [31]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29365 entries, 0 to 7089
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Report  29365 non-null  object
dtypes: object(1)
memory usage: 458.8+ KB


In [32]:
df_text.head()

,Report
0,sond nasogastr con extrem distal cam gastric . catet venos central con entrad ven yugul derech extrem distal ven cav superior . infiltr alveolar perihiliar bilateral descart edem agud pulmon .
1,epoc cardiomegali fundamental cavidad izquierd con hili prominent orig vascul . pinzamient sen costofren posterior relacion con minim derram engros . ciert grad ic .
2,derram pleural derech no vari signific con respect anterior estudi . sin cambi con respect existent cerclaj esternal . asi sign epoc insuficient cardiac .
3,condensacion lsd lid lii relacion bronconeumoni bilateral . asim apreci marc engros peribroncovascul perihili bilateral tambi sugest orig infecci .
4,condensacion lm descart primer diagnost neumoni . dad ausenci clinic infecci pacient recomiend realiz control radiolog 2 seman funcion result mism valor complet con estudi tac .


### 1.1 Data Block

In [33]:
TextBlock.from_df??

Signature:
TextBlock.from_df(
    text_cols,
    vocab=None,
    is_lm=False,
    seq_len=72,
    backwards=False,
    min_freq=3,
    max_vocab=60000,
    tok=None,
    rules=None,
    sep=' ',
    n_workers=8,
    mark_fields=None,
    tok_text_col='text',
    **kwargs,
)
Source:   
    @classmethod
    @delegates(Tokenizer.from_df, keep=True)
    def from_df(cls, text_cols, vocab=None, is_lm=False, seq_len=72, backwards=False, min_freq=3, max_vocab=60000, **kwargs):
        "Build a `TextBlock` from a dataframe using `text_cols`"
        return cls(Tokenizer.from_df(text_cols, **kwargs), vocab=vocab, is_lm=is_lm, seq_len=seq_len,
                   backwards=backwards, min_freq=min_freq, max_vocab=max_vocab)
File:      ~/fastai/fastai/text/data.py
Type:      method


In [34]:
bs_lm = (TextBlock.from_df(text_cols='Report', is_lm=True, tok_text_col='text'))
get_x = ColReader('text')
splitter = RandomSplitter(0.1, seed=42)
db_lm = DataBlock(blocks=bs_lm,
                     get_x=get_x,
                     splitter=splitter)

### 1.2 Data Loader

In [35]:
dl_lm = db_lm.dataloaders(df_text, bs=64)

### 1.3 Training

In [36]:
# import
from fastai_ulmfit.pretrained import *
url = 'https://bit.ly/ulmfit-eswiki'
# get tokenizer
tok = tokenizer_from_pretrained(url)
# pip install fastai-ulmfit
learn_lm = language_model_from_pretrained(dl_lm, url=url, metrics=[accuracy, Perplexity()])

In [37]:
learn_lm.to_fp16()
learn_lm.fine_tune(10, 4e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.125393,3.692128,0.336586,40.130150,00:30


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.572921,3.173944,0.396533,23.901556,00:31
1,3.133802,2.791537,0.438261,16.306068,00:31
2,2.821821,2.550956,0.465381,12.819350,00:31
3,2.632972,2.427551,0.481017,11.331099,00:31
4,2.525988,2.358421,0.486860,10.574243,00:31
5,2.476035,2.314493,0.494155,10.119795,00:30
6,2.418198,2.287833,0.496619,9.853564,00:30
7,2.394120,2.275082,0.499226,9.728713,00:30
8,2.371900,2.267375,0.500360,9.654028,00:31
9,2.379321,2.266123,0.499904,9.641942,00:31


### 1.4 Testing

In [38]:
N_WORDS = 40
N_SENTENCES = 2

In [39]:
TEXT = "derram pleural derech"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

derram pleural derech sin apreci mejori signific respect estudi previ fech 28 11 2010 . afectacion interstici alveol hemitorax derech asoci component derram pleural asoci icc . no apreci imagen sug metastasis . pequen derram pleural derech . cardiomegali .
derram pleural derech sin cambi respect estudi previ fech 24 02 2009 . discret empeor radiolog respect control previ fech 22 02 2013 sin objetiv variacion signific persist afectacion pulmon bilateral predomini central con patron interstici alveol predomini perihili relacion con distress


In [40]:
TEXT = "condensacion lm descart primer"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

condensacion lm descart primer opcion diagnost . liger infiltr interstici alveol perihili bibasal perihili bilateral . sond nasogastr con extrem distal infradiafragmat . canul traqueostomi . sond nasogastr con extrem distal infradiafragmat . . . via central con acces ven subclavi derech
condensacion lm descart primer posibil proces neumon infecci . no present sign enfermed metastas pulmon estudi . cardiomegali . elongacion aortic tronc supraaort . objet infiltr intersticial difus difus . cardiomegali . elongacion aortic tronc supraaort . granulom calcific lsd .


In [41]:
TEXT = "dad ausenci clinic infecci"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

dad ausenci clinic infecci . . . indic cardiotorac limit alto normal . . . apreci pequen opac pulmon mal defin proyect camp inferior izquierd . . . sign radiolog epoc . . . aument siluet cardiac . . . herni hiat . .
dad ausenci clinic infecci . . . general sin hallazg signific . cardiomegali . pinzamient sen costofren derech . protesis valvul mitral . cardiomegali . elongacion aortic . . . tub endotraqueal bien posicion . . . catet venos central con entrad


### 1.5 Saving

In [42]:
# save fine-tuned model for classification
learn_lm.save_encoder(path/'lm')
# get text classifier learner from fine-tuned model

# 2. Multi-Label Classifier

In [43]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

### 2.1 Data Block

In [44]:
labels = ["Atelectasis", "Cardiomegaly", "Consolidation",
          "Edema", "Fracture", "No Finding", "Pleural Effusion", 
          "Pneumonia", "Pneumothorax"]

In [45]:
dl_lm.seq_len

72

In [46]:
dl_lm.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 '.',
 'con',
 'derech',
 'izquierd',
 'estudi',
 'sin',
 'pleural',
 'cambi',
 'previ',
 'derram',
 'infiltr',
 'no',
 'radiolog',
 'aument',
 'atelectasi',
 'pulmon',
 'bilateral',
 'sign',
 'relacion',
 'cardiomegali',
 'respect',
 'fech',
 'hallazg',
 'signific',
 'probabl',
 'extrem',
 'pulmonar',
 'basal',
 'apreci',
 'fractur',
 'densid',
 'condensacion',
 'epoc',
 'costal',
 'distal',
 'aortic',
 'valor',
 'elongacion',
 'superior',
 'hili',
 'central',
 'hemitorax',
 'ven',
 'sen',
 'mejori',
 'costofren',
 'infecci',
 'alter',
 'comp',
 'normal',
 'indic',
 'proces',
 'dorsal',
 'cardiotorac',
 'medi',
 'sond',
 'nasogastr',
 'clinic',
 'radiografi',
 'torax',
 'pinzamient',
 'pequen',
 'control',
 'patron',
 'bas',
 'cardiac',
 'alveol',
 'intersticial',
 'neumoni',
 'call',
 'via',
 'asoci',
 'cronic',
 'orig',
 'cav',
 'catet',
 'pers',
 'subsegmentari',
 'bibasal',
 'ni',
 'vascul',


In [47]:
dl_lm.max_vocab

60000

In [48]:
bs_cl = (TextBlock.from_df('Report', seq_len=dl_lm.seq_len, vocab=dl_lm.vocab), 
          MultiCategoryBlock(encoded=True, vocab=labels))

In [49]:
db_cl = DataBlock(blocks=bs_cl,
                      get_x=ColReader('text'),
                      get_y=ColReader(labels),
                      splitter=ColSplitter('is_valid'))

In [51]:
db_cl.splitter(df_cl)

((#22275) [7090,7091,7092,7093,7094,7095,7096,7097,7098,7099...],
 (#7090) [0,1,2,3,4,5,6,7,8,9...])

### 2.2 Data Loader

In [52]:
dl_cl = db_cl.dataloaders(df_cl)

In [53]:
dl_cl.show_batch()

,text,None
0,xxbos indic cardiotorac limit alto normal . con respect estudi previ fech 27 ener 2014 observ desaparicion derram pleural derech . pers ocupacion angul cardiofren derech . revis estudi tac torac observ ultim solicit motiv zon exist ocupacion densid tej estirp lipomat xxunk provoc abomb atelectasi subsegmentari compres lm . plante diagnost diferencial probabl proces inflamatori gras angul cardiofren sin pod descart tumoracion estirp lipomat . impresion diagnost desaparicion derram pleural derech . ocupacion angul cardiofren derech lesion estirp lipomat asoci atelectasi subsegmentari lm . podri trat etiologi inflamatori sin pod descart tumoracion estirp lipomat . valor tratamient xxunk nuev control 2 seman . not dad hallazg lesion estirp lipomat angul cardiofren derech dolor torac pacient cit consult neumologi .,Atelectasis;Cardiomegaly;Pleural Effusion
1,xxbos engros cisur menor asoci liger derram pleural . infiltr perihili derech hili derech aument taman probabl etiologi vascul sin pod descart etiologi adenopati linfagitis . densid nodul angul cardiofren izquierd . imag lineal paravertebral izquierd no correspond line paraaort podri trat masa mediastin solicit histori pacient no dispon estudi . apreci imag aere proyect siluet hepat habri destac proces hepat . intent pon contact con medic peticionari pacient xxunk valor si pued xxunk sal tac . precis complet estudi con tac xxunk .,Pleural Effusion
2,xxbos . . dat clinic esofaguectomi . neumoni . . . . . radiografi torax anter posterior . . . . impresion diagnost . . tub drenaj pleural derech situacion correct . . . via central subclavi izquierd termi . . do ven cav superior . . . tub endotraqueal con extrem 5 cm carin . . . infiltr interstici alveolar xxunk bilateral extens probabl relacion con bronconeumoni distress respiratori sin cambi signific respect estudi previ fech 04 07 2011 .,Pneumonia
3,xxbos . . . . tub traqueal bien posicion . . . sond nasogastr con extrem distal infradiafragmat . . . catet venos central con acces trav ven yugul derech extrem distal vcs . . . tub drenaj pleural nivel basal derech . . . tub drenaj pleural nivel basal izquierd pers dudos line despeg pleural izquierd con espesor maxim 4 mm con lev mejori radiolog respect control manan hoy . . . atelectasi basal izquierd secundari neumotorax sin cambi .,Atelectasis;Pneumothorax
4,xxbos sign epoc con atrap aere retroesternal retrocardiac . pequen atelectasi subsegmentari sen costofren derech . aort elong . aument cifosis dorsal con calcificacion anterior relacion con enfermed bas conoc . osteopeni . c . lumb pelvis osteopeni generaliz xxunk anterior cuerp vertebral osteofit localizacion anterior fundamental l4 l5 relacion con espondilitis . disminucion espaci intersomat posterior l4 5 l5 s1 esclerosis superfici articular interapofisari l4 5 l5 s1 . calcificacion aort abdominal vas iliac femoral . xxunk bilateral .,Atelectasis
5,xxbos compar con estudi previ fech 09 11 2011 . tub drenaj torac con extrem proyect camp inferior derech sin cambi . actual parec exist lev aument aireacion ambos hemitorax con posibl disminucion component derram pleural bilateral . aparent disminucion component enfisem region cervical supraclavicul derech . respect control previ llam atencion visualizacion lesion cavit 6 cm proyect camp pulmon inferior izquierd no visualiz clar estudi previ context pacient habri valor posibil nuev neumatocel . rest sin cambi .,Pleural Effusion
6,xxbos cambi pulmonar cronic . engros pleuroparenquimat apical . patron intersticial micronodul hemitorax derech . visualiz nivel lsi nivel apic posterior existent aument densid aproxim 8 cm taman relacion primer opcion diagnost descart neoplasi sin pod descart opcion infecci si correspond con clinic . aument line paravertebral izquierd si bien podri relacion con elongacion aort no pued descart existent adenopati . complet estudi con tc torax form regl . dad pacient urgenci remit peticion realiz tc torax .,Pneumonia
7,xxbos sond nasogastr con

### 2.3 Training

In [54]:
text_classifier_learner??

Signature:
text_classifier_learner(
    dls,
    arch,
    seq_len=72,
    config=None,
    backwards=False,
    pretrained=True,
    drop_mult=0.5,
    n_out=None,
    lin_ftrs=None,
    ps=None,
    max_len=1440,
    y_range=None,
    loss_func=None,
    opt_func=<function Adam at 0x7fcfce1a40e0>,
    lr=0.001,
    splitter=<function trainable_params at 0x7fcfe7a273b0>,
    cbs=None,
    metrics=None,
    path=None,
    model_dir='models',
    wd=None,
    wd_bn_bias=False,
    train_bn=True,
    moms=(0.95, 0.85, 0.95),
)
Source:   
@delegates(Learner.__init__)
def text_classifier_learner(dls, arch, seq_len=72, config=None, backwards=False, pretrained=True, drop_mult=0.5, n_out=None,
                            lin_ftrs=None, ps=None, max_len=72*20, y_range=None, **kwargs):
    "Create a `Learner` with a text classifier from `dls` and `arch`."
    vocab = _get_text_vocab(dls)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferre

In [55]:
loss_func = BCEWithLogitsLossFlat(thresh=0.8)
metrics = [partial(accuracy_multi, thresh=0.8), 
           F1ScoreMulti(average='macro'),
           PrecisionMulti (average='macro'),
           RecallMulti (average='macro'),
           RocAucMulti (average='macro')]

In [56]:
learn_cl = text_classifier_learner(dl_cl, AWD_LSTM, metrics=metrics, loss_func=loss_func)

In [57]:
# get text classifier learner from fine-tuned model
learn_cl.load_encoder(path/'lm');

In [58]:
learn_cl.to_fp16()
learn_cl.fine_tune(10)

epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.222842,0.119095,0.950008,0.687835,0.756402,0.650388,0.934053,00:10


epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.135457,0.075848,0.967936,0.745519,0.803289,0.718301,0.962952,00:13
1,0.098330,0.059329,0.976759,0.780132,0.904921,0.758572,0.972005,00:13
2,0.076426,0.044116,0.983482,0.839504,0.925272,0.808098,0.979122,00:13
3,0.057784,0.036400,0.987854,0.874116,0.953548,0.840638,0.982721,00:13
4,0.045012,0.030080,0.989813,0.903882,0.952961,0.874758,0.989951,00:13
5,0.043131,0.029943,0.990550,0.910433,0.971144,0.877355,0.984801,00:13
6,0.041291,0.027973,0.991756,0.915610,0.958028,0.889410,0.988198,00:13
7,0.043406,0.027527,0.991662,0.917431,0.964097,0.886890,0.989567,00:13
8,0.038322,0.026316,0.991898,0.920847,0.965506,0.890116,0.989538,00:13
9,0.033709,0.027658,0.991631,0.915330,0.965503,0.885477,0.987744,00:13


### 2.4 Testing

In [59]:
learn_cl.validate()

(#6) [0.02765773795545101,0.9916311502456665,0.9153299612395295,0.9655025283050749,0.8854773955133822,0.987744068627777]

In [60]:
df_test = pd.read_csv(path/'labels.csv')

In [61]:
df_test.head()

,ImageID,Report,Atelectasis,Cardiomegaly,Consolidation,Edema,Fracture,No Finding,Pleural Effusion,Pneumonia,Pneumothorax,is_valid
0,20536686640136348236148679891455886468_k6ga29.png,sin hallazg patolog edad pacient .,0,0,0,0,0,1,0,0,0,True
1,3137231742710829928-247610802266403640553_km0qf9.png,radiografi actual comp con previ 26 juni persistent derram pleural derech . pequen atelectasi subsegmentari bas izquierd . sutur cerclaj esternotomi medi . engros pleural apical izquierd .,1,0,0,0,0,0,1,0,0,True
2,313723174271082992847610802266403640553-2_40kxq3.png,derram pleural bilateral predomini derech . pequen infiltr interstici alveolar parch bilateral ambos hemitorax . tub endotraqueal 5 mm carin . cerclaj esternotomi medi . sin grand cambi con respect estudi previ ayer .,0,0,0,0,0,0,1,0,0,True
3,313723174271082992847610802266403640553_w8dk8c.png,derram pleural bilateral predomini derech . pequen infiltr parch ambos hemitorax . tub endotraqueal . cerclaj esternotomi medi .,0,0,0,0,0,0,1,0,0,True
4,313723174271082992847610802266403640553_5_2icww3.png,sutur esternotomi medi . elevacion hemidiafragm izquierd present estudi previ . pers derram pleural bilateral mayor cuanti derech . liger infiltr perihili bibasal valor icc .,0,0,0,0,0,0,1,0,0,True


In [62]:
# labels = ["Atelectasis", "Cardiomegaly", "Consolidation",
#           "Edema", "Fracture", "No Finding", "Pleural Effusion", 
#           "Pneumonia", "Pneumothorax"]

In [63]:
# correct: 0, 0, 0, 0, 0, 0, 1 ,0, 0
learn_cl.predict("""derram pleural bilateral predomini derech . 
                pequen infiltr parch ambos hemitorax . tub endotraqueal . 
                cerclaj esternotomi medi """)

((#1) ['Pleural Effusion'],
 tensor([False, False, False, False, False, False,  True, False, False]),
 tensor([9.8117e-04, 6.2389e-04, 2.2344e-03, 4.6191e-04, 1.5978e-03, 1.5237e-04,
         9.9992e-01, 1.0890e-04, 2.0825e-04]))

In [64]:
# correct: 1, 0, 0, 0, 0, 0, 1 ,0, 0
learn_cl.predict("""radiografi actual comp con previ 26 juni persistent 
                derram pleural derech . pequen atelectasi subsegmentari bas 
                izquierd . sutur cerclaj esternotomi medi . engros pleural apical 
                izquierd . .""")

((#2) ['Atelectasis','Pleural Effusion'],
 tensor([ True, False, False, False, False, False,  True, False, False]),
 tensor([9.9685e-01, 1.1469e-03, 1.1292e-03, 3.6118e-04, 4.1340e-03, 3.3930e-04,
         9.9940e-01, 3.3115e-03, 2.1654e-04]))

### 2.5 Interpretation

In [65]:
interp = Interpretation.from_learner(learn_cl)

In [66]:
interp.plot_top_losses(5)

,input,target,predicted,probability,loss
0,xxbos mejori radiolog opac con nivel hidroaere perd volum hemitorax derech relacion con hidroneumotorax consolidacion asoci atelectasi con respect estudi realiz dia 28 12 2015 . atelectasi laminar camp medi hemitorax derech . pinzamient sen costofren izquierd . tub xxunk xxunk derech con extrem distal bas pulmon .,Atelectasis;Consolidation;Pleural Effusion;Pneumothorax,Atelectasis,0.9996416568756104,1.8113144636154175
1,xxbos neumoni resuelt sin secuel .,No Finding,Pneumonia,0.9941118359565735,1.6772671937942505
2,xxbos rx torax p hallazg signific . l,Cardiomegaly,No Finding,0.9990527033805847,1.397165298461914
3,xxbos comparacion con anterior estudi objet existent neumotorax mas 30 condicion colaps parenquim pulmon ventil existent estudi previ . aument derram pleural . liger desviacion mediastin mayor estudi previ . not comunic hallazg con min guardi,Atelectasis;Pleural Effusion;Pneumothorax,Pleural Effusion,0.9998756647109985,1.3788905143737793
4,xxbos anomali arco sext septim posterior derech . rest sin alter .,No Finding,Fracture,0.9937100410461426,1.364175796508789
